In [ ]:
DATA='data'
REGISTER='driving_log.csv'
REPO='repo'

In [ ]:
import os
import sys
import csv

In [ ]:
import operator
import copy
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

class Register:
    def _get_dialect(self, sample):
        dialect = csv.Sniffer().sniff(sample)
        dialect.skipinitialspace = True
        dialect.strict = True
        return dialect
        
    def _read_header(self, file, sample):
        return next(csv.reader(file)) if csv.Sniffer().has_header(sample) else None
        
    def __init__(self, path, prefix=''):
        self.prefix = prefix
        
        with open(os.path.join(prefix, path), 'r') as f:
            sample = f.read(4096)
            f.seek(0)
        
            self.header = self._read_header(f, sample)
            self._data = list(csv.reader(f, dialect=self._get_dialect(sample)))
            
    def split(self, fraction):
        data = self._data
        self._data = []
        first = copy.deepcopy(self)
        second = copy.deepcopy(self)
        self._data = data
        
        first._data, second._data = train_test_split(self._data, test_size=fraction)
        return first, second
        
    def __len__(self):
        return len(self._data)
        
    def shuffle(self):
        self._data = shuffle(self._data)
        
    def read(self, *args):
        getter = operator.itemgetter(*args) if args else None
        for l in self._data:
            yield getter(l) if getter else l

In [ ]:
reg = Register(REGISTER, prefix=DATA)
print('Driving log contains - {} examples'.format(len(reg)))

train_reg, valid_reg = reg.split(0.4)
valid_reg, test_reg = valid_reg.split(0.5)
print('Train set - {}\nValidation set - {}\nTest set - {}'.format(
    len(train_reg), len(valid_reg), len(test_reg)))

In [ ]:
h = reg.header
print("Header: {}".format(", ".join(h)))

center = h.index('center') if h else 0
left = h.index('left') if h else 1
right = h.index('right') if h else 2
steering = h.index('steering') if h else 3

In [ ]:
import os
import hashlib
import shutil

class ImageRepository:
    def __init__(self, location):
        self._location = location
        os.makedirs(location, exist_ok=True)
    
    def _hash(self, name):
        return hashlib.sha256(str.encode(name)).hexdigest()
        
    def _build_directories(self):
        for l1 in range(256):
            for l2 in range(256):
                os.makedirs(os.path.join(self._location, '%02x' % l1, '%02x' % l2),
                            exist_ok=True)        
    def target(self, name):
        h = self._hash(name)
        return os.path.join(self._location, h[0:2], h[2:4], h[4:])

    
    def build(self, register):
        self._build_directories()
        for name in (name 
                     for names in register.read(left, center, right)
                     for name in names):
            dst = self.target(name)
            if not os.path.exists(dst):
                shutil.copyfile(os.path.join(register.prefix, name), dst)

    def read(self, name):
        return cv2.cvtColor(cv2.imread(self.target(name)), cv2.COLOR_BGR2RGB)

In [ ]:
ir = ImageRepository(os.path.join(DATA, REPO))
ir.build(reg)

In [ ]:
import cv2

In [ ]:
def detect_lanes(gray):
    canny_low = 180
    canny_high = 500
    canny = cv2.Canny(gray, canny_low, canny_high)
    
    rho = 1
    theta = math.pi / 180
    threshold = 40
    min_line_len = 45
    max_line_gap = 15
    hough = cv2.HoughLinesP(canny, rho, theta, threshold, 
                            np.array([]), 
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    if hough is None:
        hough = []
    
    lines = np.zeros((canny.shape[0], canny.shape[1]), dtype=np.uint8) 
    for x1,y1,x2,y2 in ((x1,y1,x2,y2) 
                        for line in hough if hough is not None 
                        for x1,y1,x2,y2 in line):
        cv2.line(lines, (x1, y1), (x2, y2), color=(255,), thickness=3)

    return lines

In [ ]:
def square(img):
    transpose = img.transpose([1,0,2])
    h, w = (img.shape[i] for i in (0, 1))
    delta = 1*(w - h)/2
    mask = np.ones(len(transpose), dtype=bool)
    mask[range(int(w/2 - delta), int(w/2 + delta))] = False
    return transpose[mask].transpose([1,0,2])

In [ ]:
def resize(img, target):
    return np.dstack((cv2.resize(i, target, interpolation=cv2.INTER_NEAREST) 
                      for i in np.dsplit(img, img.shape[-1])))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [ ]:
plt.imshow(resize(square(ir.read(next(reg.read(center)))), (32, 32))[:,:,0], cmap='gray')

In [ ]:
from itertools import islice
import math

plt.figure(figsize=(16, 8))
num = 15
cols = 3
reg.shuffle()
for idx,x in enumerate(islice(reg.read(left,center,right), int(num/3))):
    for jdx,path in enumerate(x):
        img = ir.read(path)
        img = np.dstack((img, np.expand_dims(detect_lanes(img), -1)))
        img = resize(square(img), (32, 32))
        #print(img.shape)
        plt.subplot(num/cols+(1 if (num%cols) else 0), cols, 3*idx + jdx + 1)
        plt.title(path)
        plt.imshow(img[:,:,3], interpolation='nearest', cmap='gray')
plt.tight_layout()

In [ ]:
import functools

def generate(gen, fn):
    for i, s in (generated for args in gen for generated in fn(*args)):
        yield i, s
    
def generator(reg, ir, X_pipeline, y):    
    X, p = zip(*X_pipeline)
    for i, s in (generated
                 for inputs in reg.read(*X, y)
                 for f, x in zip(p, inputs[:-1])
                 for generated in 
                     functools.reduce(generate, (read_data,) + f, ((ir, x, y) for _ in range(1)))
                ):
        yield i, s

In [ ]:
def read_data(ir, name, s):
    yield ir.read(name), float(s)

def crop_image(image, s, top, bottom):
    yield image[top:bottom], s
    
def convert_colorspace(image, s):
    yield cv2.cvtColor(image, cv2.COLOR_RGB2YUV), s
    
def lanes(image, s):
    yield np.dstack((image, detect_lanes(image[:,:,0]))), s

def square_image(image, s):
    yield square(image), s
        
def resize_image(image, s, target):
    yield resize(image, target), s
    
def adjust_angle(image, s, d):
    yield image, s + d

def flip(image, s):
    yield image, s
    yield cv2.flip(image, 1), -s

In [ ]:
def batch_generator(gen, batch_size):
    while True:
        batch = tuple(np.array(x) for x in zip(*islice(gen, batch_size)))
        if len(batch) == 0: return
        assert(len(batch) == 2)    
        yield batch
        if len(batch[0]) != batch_size: return

def shuffle_batch(gen):
    for batch in gen:
        yield shuffle(*batch)
        
def infitnite_generator(generate, shuffle):
    while True:
        shuffle()
        for i,v in enumerate(generate()):
            yield v

In [ ]:
def train_generator(reg, ir):
    crop = lambda i, s: crop_image(i, s, 65, 135)
    resize = lambda i, s: resize_image(i, s, (32, 32))
    center_processor = (center, (flip,)) #(crop, convert_colorspace, lanes, square_image, flip, resize))
    left_processor = (left, (lambda i, s: adjust_angle(i, s, 5), flip))
    right_processor = (right, (lambda i, s: adjust_angle(i, s, -5), flip))
    return generator(reg, ir, (center_processor, left_processor, right_processor), steering)

In [ ]:
x = 0
limit = 8
plt.figure(figsize=(16,8))
for i,m in islice(train_generator(reg, ir), limit):
    for j in range(i.shape[-1]):
        plt.subplot(i.shape[-1], limit, x + limit*j + 1);
        plt.imshow(i[:,:,j], cmap='gray', interpolation='lanczos')
    x += 1
plt.tight_layout()

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Lambda, Conv2D, MaxPooling2D

shape = next(train_generator(reg, ir))[0].shape
batch_size=128
print('steps - ', int(len(train_reg)/float(batch_size) + 0.5))
tgen = lambda reg, ir, batch_size: infitnite_generator(
    lambda: shuffle_batch(
            batch_generator(
                train_generator(reg, ir), 
                batch_size)),
    lambda: reg.shuffle())

#vgen = lambda reg, ir, batch_size: batch_generator(train_generator(reg, ir), batch_size)

#inp = Input(shape=shape)
#print(inp)
model = Sequential()
print(model.outputs)
#model.add(Flatten(input_shape=shape))
print(model.outputs)
model.add(Conv2D(16, (5, 5), input_shape=shape))
model.add(MaxPooling2D((3, 3), (2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (5, 5)))
model.add(MaxPooling2D((3, 3), (2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (5, 5)))
model.add(MaxPooling2D((3, 3), (2, 2)))
model.add(Activation('relu'))
print(model.outputs)
model.add(Flatten())
model.add(Dense(128, activation='relu', use_bias=True))
print(model.outputs)
model.add(Dense(128, activation='relu', use_bias=True))
print(model.outputs)
model.add(Dense(1, activation='relu', use_bias=True))
print(model.outputs)

model.compile(loss='mse', optimizer='adam')#, metrics=['mae'])
model.fit_generator(generator=tgen(train_reg, ir, batch_size), 
                    steps_per_epoch=int(6*len(train_reg)/float(batch_size) + 0.5), 
                    epochs=2,
                    validation_data=tgen(valid_reg, ir, batch_size),
                    validation_steps=int(6*len(valid_reg)/float(batch_size) + 0.5),
                    verbose=1, 
                    use_multiprocessing=True)
model.save_weights('model.hdf5')
del model